# Data Importing, Cleaning, and Processing

In this project I import test cricket dataset from https://cricsheet.org/. The data contains match records from 2009 and onwards. The mens ODI data contains 1447 csv files as retrieved on 2009/09/26.

files are save in test_csv_male sub folder.

## Data Loading

First the data needs to be loaded. It is contained within a CSV files, however this CSV files themselves are not properly formatted. Within each file there are a number of lines of general match information before the actual table starts. The table itself gives a ball by ball breakdown of the match.

While the stats are quite comprehensive in terms of the number of matches and the various ball by ball breakdowns one major drawback is a lack of attributing fielding wickets. Because of this it is not possible to accurately attribute catches or runouts to individual players. Despite this many interesting insights can be gained through this extensive dataset.

The csv files from cricksheet are saved in csv format, but do not contain standard columns. Opening one in notepad each csv file seems to compose two parts. The first part contains information about the match. This includes dates, umpires, results etc. The second part of the file contains a ball by ball breakdown of the match. This includes bowler, runs scored, wickets taken etc.

To process each of these csv files I first open them as text documents. The match information and ball by ball breakdown are seperated and stored seperately. I keep the filenames as a "match id" which can be used to subsequently join datasets of corresponding matches.



In [2]:
#basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#used to convert .csv to .txt
import csv

#used to search filetypes within a directory
import glob, os

#used to handle regular expressions
import re


In [3]:
# load in each csv file and store the match information with a _header appended and the ball 
# by ball analysis with _stats appended


directory = 'odi_csv_male/'


def create_txt_files(file):
    """
    Creates two txt files from the corrupted csv file. The first file is a header file 
    containing two columns of match info. The second file is a match stats file which gives
    a ball by ball breakdown of play.
    """
    csv_file = file
    txt_file1 = file[:-4] + '_header'#remove csv extension
    txt_file2 = file[:-4] + '_stats' #split into header and match stats
    
    with open(txt_file1, 'w') as header_output_file:
        
        with open(txt_file2, 'w') as stats_output_file:
        
            with open(csv_file, 'r') as my_input_file:
                
                #using a regular expression find the file number and 
                #append a match number to the header file
                match_number = re.findall('\d+', file)[0]
                header_output_file.write("match_id#" + match_number + '\n')
                
                
                #read file row by row
                #split header and match stats
                for row in csv.reader(my_input_file):
                    if row[0] == 'info':
                        
                        #write the header file, exluding 'info list'
                        header_output_file.write("#".join(row[1:]) + '\n')
                    elif row[0] == 'ball':
                        #write the match stats file
                        stats_output_file.write("#".join(row) + '\n')
                
                
                
                #write txt_file to memory
                ##[my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
        stats_output_file.close()
    header_output_file.close()  
    return
    

# generates a text file for both the header and ball by ball information
# for each cricksheet csv file

for file in glob.glob(directory + '*.csv'):
    create_txt_files(file)
    

Now that the header and match stats info have been split from the raw csv files I will attempt to consolodate these into two new csv files containing the complete dataset.


Now that the header info and stats info have been split from the original csv files I consolodata all _stats and all _header files into two larger csv files. 


### consolodating _header csv files

Column names are set and duplicate values are kept as new columns with a _2, _3, _4... suffix appended. This is required because some matches have 3 umpires (eg. in the case one is injured and a replacement is introduced) and this means that those matches have more columns in the _header than reqular games.

In [12]:
#create an empty dataframe to store the match information from header files
match_information = pd.DataFrame()

def form_rows(df,match_information):
    """
    Transforms the input header information into a dataframe with a single row.
    """
    df = df.T
    df.columns = df.iloc[0]
    df.drop(0, inplace=True)
    df.set_index('match_id', inplace=True)
    
    
    #sort the dataframe so only the start date and finish date are kept
    #resolves issue of multiple (and variable number of) 'date' columns
    dates_filter = df.columns == 'date'
    dates = df.loc[:,dates_filter]
    df = df.loc[:,~dates_filter]
    df['start_date'] = dates.iloc[:,0]
    df['end_date'] = dates.iloc[:,-1] 
    
    #ensure that all columns are unique. Non-unique columns are appended with an integer
    df = df_column_uniquename(df)
    
    #append the new data to match_informaiton
    match_information = pd.concat([df, match_information],axis=0,ignore_index=False,sort=False)
    
    return match_information

def df_column_uniquename(df):
    """
    Transforms column names in a dataframe such that any duplicate names are made
    unique via appending a number to them
    """
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 1 #start with 2 for duplicates
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

for file in glob.glob(directory + '*header*'):
    df = pd.read_csv(file, sep='#',header=None, engine='python')
    match_information = form_rows(df,match_information)
    
#rename home and away teams    
match_information.rename(columns={'team':'home_team'}, inplace=True)
match_information.rename(columns={'team_2':'away_team'}, inplace=True)
match_information.reset_index(inplace=True)

match_information.head()

,match_id,home_team,away_team,gender,season,competition,match_number,venue,city,toss_winner,...,series,player_of_match,tv_umpire,neutralvenue,outcome,method,player_of_match_2,reserve_umpire_2,eliminator,match_number_2
0,997995,Scotland,United Arab Emirates,male,2016,"ICC World Cricket League Championship, 2015-20...",28,"Grange Cricket Club Ground, Raeburn Place",Edinburgh,United Arab Emirates,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,997993,Scotland,United Arab Emirates,male,2016,"ICC World Cricket League Championship, 2015-20...",26,"Grange Cricket Club Ground, Raeburn Place",Edinburgh,United Arab Emirates,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,997967,Ireland,Afghanistan,male,2016,NaN,5,"Civil Service Cricket Club, Stormont",Belfast,Afghanistan,...,Afghanistan in Ireland ODI Series,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,997965,Ireland,Afghanistan,male,2016,NaN,4,"Civil Service Cricket Club, Stormont",Belfast,Afghanistan,...,Afghanistan in Ireland ODI Series,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,997961,Ireland,Afghanistan,male,2016,NaN,2,"Civil Service Cricket Club, Stormont",Belfast,Ireland,...,Afghanistan in Ireland ODI Series,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#append match result to winner column in the case where there is no winner

match_information['winner'][pd.isna(match_information['winner'])] = match_information['outcome'][pd.isna(match_information['winner'])]
match_information.drop(columns='outcome', inplace=True)

### Datatypes

Now that I have organised the _header csv files into a single match__information  dataframe I can set datatypes. Since all the values were read in from a txt file the default filetypes are all object.

In [9]:
#Set datatypes for the match_information dataframe

#float types
floats = ['winner_runs','winner_wickets','match_number']
match_information[floats] = match_information[floats].astype('float16')

#dates
match_information['start_date'] = pd.to_datetime(match_information['start_date'])
match_information['end_date'] = pd.to_datetime(match_information['end_date'])

#categories
match_information['toss_decision'] = match_information['toss_decision'].astype('category')

#set toss winner to category home_team / away_team
match_information['home_toss_win'] = (match_information['toss_winner'] == match_information['home_team']).astype('category')

#likewise set winner to category home_team / away_team / draw / tie
index_home = match_information['home_team'] == match_information['winner']
#index_draw = match_information['winner'] == 'draw'
index_away = match_information['away_team'] == match_information['winner']

match_information['winner_cat'] = match_information['winner'] #copy across draw and tie results
match_information['winner_cat'].loc[index_home] = 'Home'#overwrite home team wins
match_information['winner_cat'].loc[index_away] = 'Away'#overwrite away team wins
#convert to a categorical data type
match_information['winner_cat'] = match_information['winner_cat'].astype('category')

#view datatypes
display(match_information.dtypes)

match_information.reset_index(inplace=True, drop=True)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


home_team                    object
away_team                    object
gender                       object
season                       object
competition                  object
match_number                float16
venue                        object
city                         object
toss_winner                  object
toss_decision              category
umpire                       object
umpire_2                     object
reserve_umpire               object
match_referee                object
winner                       object
winner_wickets              float16
start_date           datetime64[ns]
end_date             datetime64[ns]
winner_runs                 float16
series                       object
player_of_match              object
tv_umpire                    object
neutralvenue                 object
method                       object
player_of_match_2            object
reserve_umpire_2             object
eliminator                   object
match_number_2              

### consolodating _stats csv files


Using the _stats csv filess I now want to create two dataframes for batting statistics and bowling statistics. A match_id will be added to statistics from each match.

In [19]:
#Set column names for the dataframe
col = ['action', 'innings', 'over', 'batting_team', 'strike_batsman', 'offstrike_batsman', 'bowler', 'runs', 'extras', 'dismissal_type', 'batsman_dismissed']
ind = list(range(0,11))
header = dict(zip(ind,col))

#initialise empty dataframes to store data in
batting_stats = pd.DataFrame()
bowling_stats = pd.DataFrame()
ball_by_ball_stats = pd.DataFrame()

#def wkt_breakdown(x):
#    """
#    takes an input array and returns a count of each type of dismissal
#    """
#    #modes of dismissal for a bowler
#    return [np.sum(x=='bowled'),np.sum(x=='lbw'),np.sum(x=='caught'),np.sum(x=='caught and bowled'),np.sum(x=='stumped')]

#functions which return a tally of the mode of dismissal
def bowled(x):
    x = pd.Series(x)
    return np.sum(x=='bowled')
def lbw(x):
    x = pd.Series(x)
    return np.sum(x=='lbw')
def caught(x):
    x = pd.Series(x)
    return np.sum(x=='caught')
def c_and_b(x):
    x = pd.Series(x)
    return np.sum(x=='caught and bowled')
def stumped(x):
    x = pd.Series(x)
    return np.sum(x=='stumped')
    
#functions which return the number of runs matching a criteria
def zeros(x):
    return np.sum(x==0)
def ones(x):
    return np.sum(x==1)
def twos(x):
    return np.sum(x==2)
def threes(x):
    return np.sum(x==3)
def fours(x):
    return np.sum(x==4)
def fives(x):
    return np.sum(x==5)
def sixes(x):
    return np.sum(x==6)


def match_stats(df,bowling_stats,batting_stats,ball_by_ball_stats,match_id,match_information):

#####Generate player statistics by innings####



    #add a bowling_team column to df
    
    #get unique teams for the match
    #unique teams are retrieved from _header since some matches are too short for both teams to bat. Hence _stats
    # contains only one team, the batting team, and the bowling team remains unknown 

    match_information[['home_team','away_team']]#.loc[match_information['match_id']==296909]
    x = match_information.loc[match_information['match_id']==match_id]
    y = np.array(x[['home_team','away_team']])
    
    #initially set bowling team to batting team
    df['bowling_team'] = df['batting_team']
    #map batting team to its compliment for bowling team
    #remapper = {unique_teams[0]:unique_teams[1], unique_teams[1]:unique_teams[0]}
    remapper = {y[0,1]: y[0,0], y[0,0]: y[0,1]}
    #remap bowling team
    df['bowling_team'] = df['bowling_team'].replace(remapper)
    


###BATTING STATISTICS

#contains batsman and innings with runs, balls faced and not_out
    
    runs = df.groupby(['strike_batsman','innings'])['runs'].agg(['sum', 'count', zeros, ones, twos, threes, fours, fives, sixes])

    runs.reset_index(inplace=True)
    runs.rename(columns={'strike_batsman':'batsman','sum':'runs','count':'balls_faced'},inplace=True)
    #fill na counts

#generate an 'outs' dataframe
    outs = df[['innings','dismissal_type','strike_batsman','offstrike_batsman', 'batsman_dismissed','bowler']].dropna()
    outs.rename(columns={'batsman_dismissed':'batsman'},inplace=True)

#want to join 'outs' and 'runs' on innings and batsman
    batting = pd.merge(runs, outs, how='left', on=['batsman','innings'])


    
#remove bowler names from runouts

    batting.loc[batting.dismissal_type == 'run out','bowler'] = ''
    batting['out'] = ~pd.isna(batting.dismissal_type) #boolean for whether a batsman is out
    batting.fillna('', inplace=True) #fill remaining NaNs with empty strings

#generate and append batting position

#get a list of batsmen by appearance (batting order or position)
#interleave strike and offstrike batsmen then sort by unique values
    order = df[['innings','over','strike_batsman']]
    order.rename(columns = {'strike_batsman':'offstrike_batsman'},inplace=True)
    order = order.append(df[['innings','over','offstrike_batsman']])
    order.rename(columns = {'offstrike_batsman':'batsman'},inplace=True)
    order = order.sort_values(by = ['innings','over'])
    order = order.reset_index(drop=True)
    order = order.drop('over',axis=1)
#now that we have a sorted list of batsmen need to extract a list of unique batsman rows
    order = order.drop_duplicates()
    order['batting_pos'] = order.groupby('innings').cumcount() + 1

#merge the batting dataframe with the batting order dataframe
    batting = pd.merge(batting,order,how='left',on=['batsman','innings'])
        #sort by the innings and batting order for a more readable output
    batting = batting.sort_values(by = ['innings','batting_pos'])
    batting = batting.reset_index(drop=True)
 
 #join the batting team to batsman
    names_and_teams = df[['strike_batsman','batting_team']].drop_duplicates()
    batting = pd.merge(batting, names_and_teams, how='left', left_on='batsman', right_on='strike_batsman')
    





###BOWLING STATISTICS

    wkt_types = batting[['innings','bowler','dismissal_type']]  
    #wkt_types = wkt_types.groupby(['innings','bowler'])['dismissal_type'].agg(wkt_breakdown)
    wkt_types = wkt_types.groupby(['innings','bowler'])['dismissal_type'].agg([bowled,lbw,caught,c_and_b,stumped])
    wkt_types = wkt_types.reset_index(level=[0])   
    wkt_types.reset_index(inplace=True)
    wkt_types.rename(columns={'dismissal_type':'b / lbw / c / c&b / st'},inplace=True)

    deliveries = df.groupby(['bowler','innings'])['runs'].agg(['sum','count'])
    deliveries.reset_index(inplace=True)
    deliveries.rename(columns={'sum':'runs','count':'deliveries'},inplace=True)

    wickets = batting[['innings','bowler']] #'bowler'  below
    wickets = wickets.groupby(['innings','bowler'])['bowler'].agg(['count'])
    wickets.rename(columns={'count':'wickets'},inplace=True)

    wickets = wickets.reset_index(level=[0])
    wickets.rename(columns={'bowler':'wickets'},inplace=True)
    wickets.reset_index(inplace=True)

#join wickets to deliveries

    bowling = pd.merge(deliveries, wickets, how='left', on=['bowler','innings'])
    

#join bowling to wkt_types
    bowling = pd.merge(bowling, wkt_types, how='left', on=['bowler','innings'])

#fill NaN entries with zeros.
    bowling.fillna(0,inplace=True)

     #join the bowling team to bowling
        
    names_and_teams = df[['bowler','bowling_team']].drop_duplicates()
    bowling = pd.merge(bowling, names_and_teams, how='left', on='bowler')
        
#Append match_id to 'bowling' and 'batting'

    bowling['match_id'] = match_id
    batting['match_id'] = match_id
    
#Generate complete ball by ball stats
    
    df['match_id'] = match_id
    ball_by_ball_stats = pd.concat([ball_by_ball_stats, df], axis=0,ignore_index=False,sort=False)
    
    
    
#Append 'bowling' and 'batting' to 'bowling_stats' and 'batting_stats'
    
    bowling_stats = pd.concat([bowling, bowling_stats],axis=0,ignore_index=False,sort=False)
    batting_stats = pd.concat([batting, batting_stats],axis=0,ignore_index=False,sort=False)

    return bowling_stats, batting_stats, ball_by_ball_stats
    
    
    
### Loop through files and execute

# NEED TO APPEND MATCH NUMBER IN THE LOOP ABOVE

for file in glob.glob(directory + '*stats*'):
    df = pd.read_csv(file, sep='#',header=None, engine='python')
    df.rename(columns=header, inplace=True)
    #run a loop procedure
    match_id = re.findall(r'\d+', file)
    match_id = match_id[0]
    bowling_stats, batting_stats, ball_by_ball_stats = match_stats(df,bowling_stats,batting_stats,ball_by_ball_stats,match_id, match_information)

bowling_stats.reset_index(inplace=True)
batting_stats.reset_index(inplace=True)
ball_by_ball_stats.reset_index(inplace=True)


In [20]:
##display some bowling_stats and batting_stats and confirm that they are in working order
display(ball_by_ball_stats.head())

display(bowling_stats.head())

display(batting_stats.head())

,index,action,innings,over,batting_team,strike_batsman,offstrike_batsman,bowler,runs,extras,dismissal_type,batsman_dismissed,bowling_team,match_id
0,0,ball,1,0.1,Australia,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,Pakistan,1000887
1,1,ball,1,0.2,Australia,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,Pakistan,1000887
2,2,ball,1,0.3,Australia,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,Pakistan,1000887
3,3,ball,1,0.4,Australia,DA Warner,TM Head,Mohammad Amir,0,0,NaN,NaN,Pakistan,1000887
4,4,ball,1,0.5,Australia,DA Warner,TM Head,Mohammad Amir,0,1,NaN,NaN,Pakistan,1000887


,index,bowler,innings,runs,deliveries,wickets,bowled,lbw,caught,c_and_b,stumped,bowling_team,match_id
0,0,AC Evans,1,47,53,2.0,0.0,0.0,2.0,0.0,0.0,Scotland,997995
1,1,Ahmed Raza,2,46,60,0.0,0.0,0.0,0.0,0.0,0.0,United Arab Emirates,997995
2,2,Amjad Javed,2,23,30,0.0,0.0,0.0,0.0,0.0,0.0,United Arab Emirates,997995
3,3,CB Sole,1,47,64,3.0,0.0,0.0,3.0,0.0,0.0,Scotland,997995
4,4,CD de Lange,1,42,60,2.0,0.0,1.0,1.0,0.0,0.0,Scotland,997995


,index,batsman,innings,runs,balls_faced,zeros,ones,twos,threes,fours,...,sixes,dismissal_type,strike_batsman_x,offstrike_batsman,bowler,out,batting_pos,strike_batsman_y,batting_team,match_id
0,0,Rohan Mustafa,1,19,20,10,6,1,1,2,...,0,caught,Rohan Mustafa,Mohammad Shahzad (2),CB Sole,True,1,Rohan Mustafa,United Arab Emirates,997995
1,1,L Sreekumar,1,16,19,14,2,0,0,2,...,1,caught,L Sreekumar,Rohan Mustafa,SM Sharif,True,2,L Sreekumar,United Arab Emirates,997995
2,2,Mohammad Shahzad (2),1,21,32,23,5,0,0,4,...,0,caught,Mohammad Shahzad (2),Shaiman Anwar,SM Sharif,True,3,Mohammad Shahzad (2),United Arab Emirates,997995
3,3,Shaiman Anwar,1,63,77,39,28,4,1,3,...,2,caught,Shaiman Anwar,Rameez Shahzad,CB Sole,True,4,Shaiman Anwar,United Arab Emirates,997995
4,4,Muhammad Usman,1,43,48,20,21,4,0,2,...,1,caught,Muhammad Usman,Shaiman Anwar,AC Evans,True,5,Muhammad Usman,United Arab Emirates,997995


## Save the output

There are three output files which will be saved to the processed_data subfolder. These are match_information, bowling_stats and batting_stats. Each of these dataframes can be linked to each other via joins on match_id.

In [21]:
###### SAVE FILES ######

#Need to save 'match_information', 'bowling_stats' and 'batting_stats'

match_information.to_csv(r'processed_data\match_information.csv')
batting_stats.to_csv(r'processed_data\batting_stats.csv')
bowling_stats.to_csv(r'processed_data\bowling_stats.csv')
ball_by_ball_stats.to_csv(r'processed_data\ball_by_ball_stats.csv')

print('finished')


finished
